In [254]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random
import numpy as np

In [255]:
input_path = 'sto/'
kg = input_path + 'transitive.nt'
dataset = pd.read_csv(kg, delimiter='---', header=None)
dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,0
0,<https://w3id.org/i40/sto#IEC_61158> <https://...
1,<https://w3id.org/i40/sto#ISO_19440> <https://...
2,<https://w3id.org/i40/sto#ISA-95> <https://w3i...
3,<https://w3id.org/i40/sto#ENISA_CloudComputing...
4,<https://w3id.org/i40/sto#DIN_EN_50129> <https...


In [256]:
g = Graph()
g.parse("sto/transitive.nt", format="nt")

qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select distinct ?s where {
            ?s ?p ?o .
    }""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [257]:
entities.shape

(249, 1)

## Select all standard triples given a specific property

In [258]:
count = 0
#prop = 'hasClassification'
prop = 'relatedTo'
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:""" +prop+ """ ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append('<'+str("%s" %row)+'>')
    
    if len(obj)>0:
        lst.o=obj
        lst.s='<'+standard+'>'
        lst.p='<https://w3id.org/i40/sto#'+prop+'>'
        list_pd = pd.concat([list_pd, lst], ignore_index=True)
    else:
        count+=1

#list_pd = list_pd.drop_duplicates().reset_index(drop=True)

In [259]:
list_pd.head()

,s,p,o
0,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_14739-1>
1,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_PAS_17506>
2,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ASME_Y1441>
3,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#ISO_15926>
4,<https://w3id.org/i40/sto#ISO_15704>,<https://w3id.org/i40/sto#relatedTo>,<https://w3id.org/i40/sto#RFC_7540>


In [260]:
list_pd.shape

(22969, 3)

## Create the test set [Select 30% of triple randomly] 

In [261]:
n_select= round(3*list_pd.shape[0]/10)
print(n_select)
index = list_pd.index.tolist()
#sampling = random.choices(index, k=n_select)
sampling = random.sample(index, k=n_select)

6891


In [262]:
test_set=list_pd.iloc[sampling]
duplicateRowsDF = test_set[test_set.duplicated()]
test_set = test_set.drop_duplicates().reset_index(drop=True)
print(test_set.shape[0])
duplicateRowsDF

6891


,s,p,o


In [ ]:
#test_set.sort_values(by='s', ascending=False)

In [263]:
test_set['dot'] = '.'

In [264]:
test_set_nt=pd.DataFrame(test_set['s'] + ' '+ test_set['p']+ ' '+ test_set['o']+ ' '+ test_set['dot'])
test_set_nt.head(20)

,0
0,<https://w3id.org/i40/sto#VDI_VDE_2182> <https...
1,<https://w3id.org/i40/sto#DIN_EN_50129> <https...
2,<https://w3id.org/i40/sto#NIST_800-144> <https...
3,<https://w3id.org/i40/sto#IEC_29182-1> <https:...
4,<https://w3id.org/i40/sto#NIST_800-144> <https...
5,<https://w3id.org/i40/sto#IEC_61850_P10_E1> <h...
6,<https://w3id.org/i40/sto#IEC_62279> <https://...
7,<https://w3id.org/i40/sto#RFC_793> <https://w3...
8,<https://w3id.org/i40/sto#IEC_61850> <https://...
9,<https://w3id.org/i40/sto#IEC_61850_P10_E1> <h...


## Create training test [difference bettwen dataset and test set]

In [265]:
training_set = dataset[~dataset.apply(tuple,1).isin(test_set_nt.apply(tuple,1))]
training_set.shape[0]

16078

In [266]:
dataset.shape[0]-training_set.shape[0]

6891

## Save training and test set

In [267]:
training_set.to_csv(input_path+'training_set_relatedTo.nt', index=None, header=None)
test_set_nt.to_csv(input_path+'test_set_relatedTo.nt', index=None, header=None)

## Cheking the training set created

In [274]:
g = Graph()
g.parse("sto/training_set_relatedTo.nt", format="nt")
    
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select distinct ?s where {
            ?s ?p ?o .
    }""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [275]:
entities

,s
0,https://w3id.org/i40/sto#IEC_61511
1,https://w3id.org/i40/sto#RFC_793
2,https://w3id.org/i40/sto#IEC_60870_P6_S503_E3
3,https://w3id.org/i40/sto#IEC_61968_P4_E1
4,https://w3id.org/i40/sto#IEEE_1686
5,https://w3id.org/i40/sto#IEC_62890
6,https://w3id.org/i40/sto#IEC_27031
7,https://w3id.org/i40/sto#IEC_62424
8,https://w3id.org/i40/sto#ISO_29100
9,https://w3id.org/i40/sto#ISO_9000
